# tensorboard with kears model - 使用tensorboard可视化keras模型的训练过程

这个例子展示了如何使用tensorboard可视化keras模型的训练过程。

- Author: 浪漫葱香
- Project: https://github.com/Romantoscalion/TensorFlow-Examples-Chinese

In [12]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np
import keras._tf_keras.keras.optimizers as optimizers

In [13]:
# MNIST dataset parameters. - 数据集超参数
num_classes = 10 # 0 to 9 digits - 数字种类, 0-9共10种
num_features = 784 # 28*28 - 图片大小, 28*28=784 个特征

# Training parameters. - 训练超参数
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50
epoch = 20

In [14]:
# Prepare MNIST data. - 准备 MNIST 数据以及数据预处理
import keras.api.datasets.mnist as mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32. - 转换为 float32 类型（原先为 uint8 类型，代表0~255的灰度值）
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28). - 将图像展平为 784 特征（28*28）
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1]. - 将图像的值从 [0, 255] 标准化到 [0, 1]
x_train, x_test = x_train / 255., x_test / 255.

In [15]:
# Use tf.data API to shuffle and batch data. - 使用 tf.data API 来打乱 并 分批数据
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

## 构建一个简单的Keras模型

Basic neural network with TF Model

以一个简单的TensorFlow神经网络为例

In [16]:
from keras import Model, layers
import keras._tf_keras.keras.optimizers as optimizers

In [17]:
# MNIST dataset parameters. - 数据集超参数
num_classes = 10 # 0 to 9 digits - 数字种类, 0-9共10种
num_features = 784 # 28*28 - 图片大小, 28*28=784 个特征

# Training parameters. - 训练超参数
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 100

In [18]:
# Create TF Model. - 创建 TF 模型
import keras


class NeuralNet(Model):
    # Set layers. - 初始化层
    def __init__(self):
        super(NeuralNet, self).__init__(name="NeuralNet")
        # First fully-connected hidden layer. - 第一个全连接隐藏层
        self.fc1 = layers.Dense(64, activation=tf.nn.relu)
        # Second fully-connected hidden layer. - 第二个全连接隐藏层
        self.fc2 = layers.Dense(128, activation=tf.nn.relu)
        # Third fully-connecter hidden layer. - 第三个全连接隐藏层
        self.out = layers.Dense(num_classes, activation=tf.nn.softmax)

    # Set forward pass. - 前向传播方法
    def __call__(self, x, is_training=False):
        x = self.fc1(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            # tf 交叉熵期望没有 softmax 的 logits，所以只在不训练时应用 softmax
            x = tf.nn.softmax(x)
        return x
    
    def creat_model(): # 使用Sequential模式创建模型
        return keras.models.Sequential([
            keras.layers.InputLayer(shape=(784,)),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(10, activation='softmax')
        ])

# Build neural network model. - 构建神经网络模型
neural_net = NeuralNet()

In [19]:
# Cross-Entropy loss function. - 交叉熵损失函数
def cross_entropy(y_pred, y_true):
    y_true = tf.cast(y_true, tf.int64)
    crossentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(crossentropy)

# Accuracy metric. - 准确率评估方法
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    # 预测结果 是 预测结果向量 中得分最高的项的索引（即 argmax）
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Adam optimizer.
optimizer = optimizers.Adam(learning_rate)

In [20]:
# Optimization process.  - 优化过程
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    # 使用梯度带自动计算梯度
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy(pred, y)

        # Compute gradients. - 计算梯度
        gradients = g.gradient(loss, neural_net.trainable_variables)

        # Update W and b following gradients. - 根据梯度更新 W 和 b
        optimizer.apply_gradients(zip(gradients, neural_net.trainable_variables))

In [21]:
# Run training for the given number of steps. - 运行给定步数的训练
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    # 运行优化器更新 W 和 b 的值
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=False)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 2.179951, accuracy: 0.953125
step: 200, loss: 2.182299, accuracy: 0.933594
step: 300, loss: 2.179307, accuracy: 0.949219
step: 400, loss: 2.181699, accuracy: 0.941406
step: 500, loss: 2.180943, accuracy: 0.937500
step: 600, loss: 2.176073, accuracy: 0.972656
step: 700, loss: 2.172978, accuracy: 0.992188
step: 800, loss: 2.176641, accuracy: 0.968750
step: 900, loss: 2.177418, accuracy: 0.960938
step: 1000, loss: 2.175576, accuracy: 0.976562


## 使用tensorboard可视化

In [22]:
model = NeuralNet.creat_model() # 使用Sequential模式创建模型
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy']) # 指定优化器、损失函数、评估方法

log_dir = "./tmp/tensorflow_logs/example/" # 日志文件保存路径
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) # 创建 TensorBoard 回调
model.fit(x=x_train, 
          y=y_train, 
          batch_size=batch_size, 
          epochs=epoch,
          callbacks=[tensorboard_callback]) # 训练模型，并指定回调，让 TensorBoard 可以监控和记录训练过程

Epoch 1/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7923 - loss: 0.7755
Epoch 2/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9380 - loss: 0.2245  
Epoch 3/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9557 - loss: 0.1606
Epoch 4/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9646 - loss: 0.1242  
Epoch 5/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9722 - loss: 0.1000  
Epoch 6/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9759 - loss: 0.0858  
Epoch 7/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9784 - loss: 0.0753  
Epoch 8/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9823 - loss: 0.0623  
Epoch 9/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9857 - loss: 0.0551  
Epoch 10/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9877 - loss: 0.0458
Epoch 11/20
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9895 - loss: 0.0405  
Epoch 12/20
235/235 ━━━━━━━━━━━━━━━━

## 查看tensorboard日志

如果你的IDE有相关的拓展，那么此时IDE应该会提示你可以直接在IDE内浏览TensorBorad日志，如果没有，你可以在命令行中输入以下命令：

tensorboard --logdir=your\log\path

随后用浏览器打开返回的本地链接即可查看TensorBoard日志。

你可以在日志中看到loss和accuracy的变化情况，以及模型结构等信息。

![tensorboard_new_1](../../../resources/img/tf2/tensorboard_new_1.png)
